## Problem 3: How many people live near shopping centers? (8 points)

In the last step of this analysis, use a *spatial join* to relate data from a population grid data set to the buffer layer created in *problem 2* to find out how many people live in all population grid cells that are **within** 1.5 km distance from each shopping centre. 

Use the same population grid data set as during [lesson 3](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/spatial-join.html) (load it directly from WFS, don’t forget to assign a CRS).


*Feel free to divide your solution into more codeblocks than prepared! Remember to add comments to your code :)*

### a) Load the population grid data set and the buffer geometries (2 points)

Use the same population grid data set as during [lesson 3](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/spatial-join.html) (load it directly from WFS, don’t forget to assign a CRS). Load the data into a `GeoDataFrame` called `population_grid`.

(optional) If you want, discard unneeded columns and translate the remaining column names from Finnish to English.

In [30]:
# ADD YOUR OWN CODE HERE
import geopandas
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

population_grid = geopandas.read_file(
    (
        "https://kartta.hsy.fi/geoserver/wfs"
        "?service=wfs"
        "&version=2.0.0"
        "&request=GetFeature"
        "&typeName=asuminen_ja_maankaytto:Vaestotietoruudukko_2020"
        "&srsName=EPSG:3879"
    ),
)
population_grid.crs = "EPSG:3879" 

In [5]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
import geopandas
import pyproj

assert isinstance(population_grid, geopandas.GeoDataFrame)
assert population_grid.crs == pyproj.CRS("EPSG:3879")



Load the buffers computed in *problem 2* into a `GeoDataFrame` called `shopping_centre_buffers`. Add an `assert` statement to check whether the two data frames are in the same CRS.

In [9]:
# ADD YOUR OWN CODE HERE
shopping_centre_buffers = geopandas.read_file('shopping_centres.gpkg', layer='buffers')

In [10]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
assert isinstance(shopping_centre_buffers, geopandas.GeoDataFrame)
assert shopping_centre_buffers.geometry.geom_type.unique() == ["Polygon"]
assert shopping_centre_buffers.crs == pyproj.CRS("EPSG:3879")


---

### b) Carry out a *spatial join* between the `population_grid` and the `shopping_centre_buffers`  (2 points)

Join the shopping centre’s `id` column (and others, if you want) to the population grid data frame, for all population grid cells that are **within** the buffer area of each shopping centre. [Use a *join-type* that retains only rows from both input data frames for which the geometric predicate is true](https://geopandas.org/en/stable/gallery/spatial_joins.html#Types-of-spatial-joins). 


In [31]:
population_grid

,gml_id,index,asukkaita,asvaljyys,ika0_9,ika10_19,ika20_29,ika30_39,ika40_49,ika50_59,ika60_69,ika70_79,ika_yli80,geometry
0,Vaestotietoruudukko_2020.1,703,5,51,99,99,99,99,99,99,99,99,99,"POLYGON ((25472499.995 6685998.998, 25472499.9..."
1,Vaestotietoruudukko_2020.2,710,8,44,99,99,99,99,99,99,99,99,99,"POLYGON ((25472499.995 6684249.004, 25472499.9..."
2,Vaestotietoruudukko_2020.3,711,5,90,99,99,99,99,99,99,99,99,99,"POLYGON ((25472499.995 6683999.005, 25472499.9..."
3,Vaestotietoruudukko_2020.4,715,13,34,99,99,99,99,99,99,99,99,99,"POLYGON ((25472499.995 6682998.998, 25472499.9..."
4,Vaestotietoruudukko_2020.5,848,5,53,99,99,99,99,99,99,99,99,99,"POLYGON ((25472749.993 6690249.003, 25472749.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5832,Vaestotietoruudukko_2020.5833,27108,7,70,99,99,99,99,99,99,99,99,99,"POLYGON ((25513249.999 6686248.997, 25513249.9..."
5833,Vaestotietoruudukko_2020.5834,27109,6,38,99,99,99,99,99,99,99,99,99,"POLYGON ((25513249.999 6685998.998, 25513249.9..."
5834,Vaestotietoruudukko_2020.5835,27110,14,23,99,99,99,99,99,99,99,99,99,"POLYGON ((25513249.999 6685748.999, 25513249.9..."
5835,Vaestotietoruudukko_2020.5836,27111,12,59,99,99,99,99,99,99,99,99,99,"POLYGON ((25513249.999 6685499, 25513249.999 6..."


In [32]:
# ADD YOUR OWN CODE HERE
# population_grid = population_grid[["asukkaita", "geometry"]]
population_grid = population_grid.rename(columns={"asukkaita": "population"})

population_grid["population_density"] = (
    population_grid["population"]
    / (population_grid.area / 1_000_000)
)

In [39]:
shopping_centres_with_population = shopping_centre_buffers.sjoin(
    population_grid,
    how='left',
    predicate='within'
)


---

### c) Compute the population sum around shopping centres (2 points)

Group the resulting (joint) data frame by shopping centre (`id` or `name`), and calculate the `sum()` of the population living inside the 1.5 km radius around them.

Print the results, for instance, in the form "12345 people live within 1.5 km from REDI".

In [54]:
# ADD YOUR OWN CODE HERE
for name, population in shopping_centres_with_population.groupby('name')['population_density'].sum().items():
    print(f'{population:.0f} people live within 1.5 km from {name}')

2128 people live within 1.5 km from Forum
10432 people live within 1.5 km from Iso-omena
0 people live within 1.5 km from Itis
0 people live within 1.5 km from Jumbo
10352 people live within 1.5 km from REDI
4512 people live within 1.5 km from Sello
8544 people live within 1.5 km from Tripla 



---

### d) Reflection

Good job! You are almost done with this week’s exercise. Please quickly answer the following short questions:
    
- How challenging did you find problems 1-3 (on scale to 1-5), and why?
- What was easy?
- What was difficult?

Add your answers in a new *Markdown* cell below: